# 📦 Import Libraries

# Loan Payback Prediction - Kaggle Competition

**Goal:** Predict the probability that a borrower will pay back their loan  
**Best Model:** CatBoost with Optuna optimization → **0.9214 AUC**

In [ ]:
# Data processing
import polars as pl
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

# Advanced models
import catboost as cb
import optuna
from pytorch_tabnet.tab_model import TabNetClassifier

# Settings
import warnings
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

In [2]:
train_df = pl.read_csv("data/train.csv")

print(f"Dataset shape: {train_df.shape}")
print(f"\nColumn names: {train_df.columns}")
print("\nFirst few rows:")
train_df.head()

Dataset shape: (593994, 13)

Column names: ['id', 'annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate', 'gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade', 'loan_paid_back']

First few rows:


id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
i64,f64,f64,i64,f64,f64,str,str,str,str,str,str,f64
0,29367.99,0.084,736,2528.42,13.67,"""Female""","""Single""","""High School""","""Self-employed""","""Other""","""C3""",1.0
1,22108.02,0.166,636,4593.1,12.92,"""Male""","""Married""","""Master's""","""Employed""","""Debt consolidation""","""D3""",0.0
2,49566.2,0.097,694,17005.15,9.76,"""Male""","""Single""","""High School""","""Employed""","""Debt consolidation""","""C5""",1.0
3,46858.25,0.065,533,4682.48,16.1,"""Female""","""Single""","""High School""","""Employed""","""Debt consolidation""","""F1""",1.0
4,25496.7,0.053,665,12184.43,10.21,"""Male""","""Married""","""High School""","""Employed""","""Other""","""D1""",1.0


# Load and Explore Data

In [ ]:
# Load all datasets
test_df = pl.read_csv("data/test.csv")
sample_submission = pl.read_csv("data/sample_submission.csv")

# Convert to pandas for easier manipulation
train_pd = train_df.to_pandas()
test_pd = test_df.to_pandas()

print("Dataset shapes:")
print(f"  Training: {train_df.shape}")
print(f"  Test: {test_df.shape}")
print(f"  Target distribution: {train_pd['loan_paid_back'].mean():.3f}")
print(f"\nNo missing values: {train_df.null_count().sum().sum() == 0}")

Test dataset shape: (254569, 12)
Sample submission shape: (254569, 2)

Test columns: ['id', 'annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate', 'gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade']
Sample submission columns: ['id', 'loan_paid_back']

Null values in train data:
shape: (1, 13)
┌─────┬────────────┬────────────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ id  ┆ annual_inc ┆ debt_to_in ┆ credit_sco ┆ … ┆ employment ┆ loan_purpo ┆ grade_sub ┆ loan_paid │
│ --- ┆ ome        ┆ come_ratio ┆ re         ┆   ┆ _status    ┆ se         ┆ grade     ┆ _back     │
│ u32 ┆ ---        ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---        ┆ ---       ┆ ---       │
│     ┆ u32        ┆ u32        ┆ u32        ┆   ┆ u32        ┆ u32        ┆ u32       ┆ u32       │
╞═════╪════════════╪════════════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 0   ┆ 0       

In [ ]:
def feature_engineering(df):
    """Create comprehensive feature engineering pipeline"""
    df_processed = df.copy()
    
    # Basic engineered features
    df_processed['loan_to_income_ratio'] = df_processed['loan_amount'] / df_processed['annual_income']
    df_processed['income_per_debt_ratio'] = df_processed['annual_income'] / (df_processed['debt_to_income_ratio'] + 1e-6)
    df_processed['credit_score_normalized'] = (df_processed['credit_score'] - 300) / (850 - 300)
    
    # Extract grade components
    df_processed['grade'] = df_processed['grade_subgrade'].str[0]
    df_processed['subgrade'] = df_processed['grade_subgrade'].str[1:].astype(int)
    df_processed['grade_numeric'] = df_processed['grade'].map({'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7})
    
    # Binning features
    df_processed['credit_score_bin'] = pd.cut(df_processed['credit_score'], 
                                            bins=[0, 580, 670, 740, 850], 
                                            labels=['Poor', 'Fair', 'Good', 'Excellent'])
    df_processed['interest_rate_bin'] = pd.cut(df_processed['interest_rate'], 
                                             bins=[0, 10, 13, 16, 25], 
                                             labels=['Low', 'Medium', 'High', 'Very_High'])
    
    # Advanced financial metrics
    df_processed['risk_score'] = (df_processed['debt_to_income_ratio'] * df_processed['interest_rate']) / (df_processed['credit_score'] / 100)
    df_processed['payment_capacity'] = df_processed['annual_income'] / (df_processed['loan_amount'] * df_processed['interest_rate'] / 100)
    df_processed['debt_burden'] = df_processed['loan_amount'] * df_processed['debt_to_income_ratio']
    df_processed['affordability_index'] = df_processed['annual_income'] / (df_processed['loan_amount'] + df_processed['annual_income'] * df_processed['debt_to_income_ratio'])
    
    # Credit and income interactions
    df_processed['credit_income_interaction'] = df_processed['credit_score'] * np.log1p(df_processed['annual_income'])
    df_processed['credit_loan_ratio'] = df_processed['credit_score'] / df_processed['loan_amount'] * 1000
    df_processed['income_stability'] = df_processed['annual_income'] / (df_processed['debt_to_income_ratio'] + 1e-6)
    df_processed['grade_risk'] = df_processed['grade_numeric'] * df_processed['interest_rate']
    df_processed['subgrade_risk'] = df_processed['subgrade'] * df_processed['debt_to_income_ratio']
    
    # Log transformations
    df_processed['log_annual_income'] = np.log1p(df_processed['annual_income'])
    df_processed['log_loan_amount'] = np.log1p(df_processed['loan_amount'])
    df_processed['sqrt_credit_score'] = np.sqrt(df_processed['credit_score'])
    
    # Categorical combinations
    df_processed['credit_employment_combo'] = df_processed['credit_score_bin'].astype(str) + '_' + df_processed['employment_status'].astype(str)
    df_processed['grade_purpose_combo'] = df_processed['grade'].astype(str) + '_' + df_processed['loan_purpose'].astype(str)
    
    return df_processed

# Apply feature engineering
train_processed = feature_engineering(train_pd)
test_processed = feature_engineering(test_pd)

print(f"Features created: {train_processed.shape[1]} total features")
new_features = set(train_processed.columns) - set(train_pd.columns)
print(f"New features: {len(new_features)}")

New features created:
   loan_to_income_ratio  income_per_debt_ratio  credit_score_normalized grade  \
0              0.086094          349614.766491                 0.792727     C   
1              0.207757          133180.041084                 0.610909     D   
2              0.343080          510986.484675                 0.716364     C   
3              0.099929          720885.063307                 0.423636     F   
4              0.477883          481060.734703                 0.663636     D   

   subgrade credit_score_bin interest_rate_bin  
0         3             Good              High  
1         3             Fair            Medium  
2         5             Good               Low  
3         1             Poor         Very_High  
4         1             Fair            Medium  


# Feature Engineering

In [ ]:
def prepare_features(df, fit_encoders=True, encoders=None):
    """Prepare features for modeling with comprehensive encoding"""
    df_model = df.copy()
    
    # High cardinality categorical features
    high_card_cats = ['credit_employment_combo', 'grade_purpose_combo']
    
    # Regular categorical features
    categorical_features = ['gender', 'marital_status', 'education_level', 'employment_status', 
                           'loan_purpose', 'grade_subgrade', 'credit_score_bin', 'interest_rate_bin', 'grade']
    
    # Numerical features
    numerical_features = ['annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 
                         'interest_rate', 'loan_to_income_ratio', 'income_per_debt_ratio', 
                         'credit_score_normalized', 'subgrade', 'risk_score', 'payment_capacity',
                         'debt_burden', 'affordability_index', 'credit_income_interaction',
                         'credit_loan_ratio', 'income_stability', 'grade_numeric', 'grade_risk',
                         'subgrade_risk', 'log_annual_income', 'log_loan_amount', 'sqrt_credit_score']
    
    if encoders is None:
        encoders = {}
    
    # Frequency encoding for high cardinality features
    for feature in high_card_cats:
        if fit_encoders:
            freq_map = df_model[feature].value_counts().to_dict()
            encoders[f'{feature}_freq'] = freq_map
            df_model[f'{feature}_freq'] = df_model[feature].map(freq_map).fillna(0)
        else:
            freq_map = encoders.get(f'{feature}_freq', {})
            df_model[f'{feature}_freq'] = df_model[feature].map(freq_map).fillna(0)
    
    # Label encoding for categorical features
    for feature in categorical_features:
        if fit_encoders:
            le = LabelEncoder()
            df_model[feature] = le.fit_transform(df_model[feature].astype(str))
            encoders[feature] = le
        else:
            if feature in encoders:
                unique_vals = set(df_model[feature].astype(str).unique())
                known_vals = set(encoders[feature].classes_)
                unseen_vals = unique_vals - known_vals
                if unseen_vals:
                    df_model[feature] = df_model[feature].astype(str).apply(
                        lambda x: x if x in known_vals else encoders[feature].classes_[0]
                    )
                df_model[feature] = encoders[feature].transform(df_model[feature].astype(str))
    
    # Select final features
    freq_features = [f'{feat}_freq' for feat in high_card_cats]
    final_features = numerical_features + categorical_features + freq_features
    
    return df_model[final_features], encoders

# Prepare features
X, encoders = prepare_features(train_processed, fit_encoders=True)
y = train_processed['loan_paid_back']
X_test, _ = prepare_features(test_processed, fit_encoders=False, encoders=encoders)

# Split for validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training features shape: {X.shape}")
print(f"Training set: {X_train.shape}, Validation set: {X_val.shape}")

Training data shape: (593994, 17)
Test data shape: (254569, 17)
Features: ['annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate', 'loan_to_income_ratio', 'income_per_debt_ratio', 'credit_score_normalized', 'subgrade', 'gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade', 'credit_score_bin', 'interest_rate_bin']
Target distribution: loan_paid_back
1.0    474494
0.0    119500
Name: count, dtype: int64


# Model Training & Optimization

In [ ]:
# CatBoost Hyperparameter Optimization with Optuna

def optimize_catboost(trial):
    """Optimize CatBoost hyperparameters using Optuna"""
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'random_strength': trial.suggest_float('random_strength', 0.1, 1.0),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'verbose': False,
        'random_seed': 42
    }
    
    model = cb.CatBoostClassifier(**params)
    model.fit(X_train, y_train, verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_pred_proba)
    return auc

print("🔍 Optimizing CatBoost with Optuna (20 trials)...")
study_catboost = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
study_catboost.optimize(optimize_catboost, n_trials=20, show_progress_bar=True)

print(f"\n✅ Best CatBoost AUC: {study_catboost.best_value:.4f}")
print(f"Best parameters: {study_catboost.best_params}")

# Train final model with best parameters
best_catboost_params = study_catboost.best_params
best_catboost_params.update({'verbose': False, 'random_seed': 42})
catboost_model = cb.CatBoostClassifier(**best_catboost_params)
catboost_model.fit(X_train, y_train, verbose=False)

# Evaluate on validation set
catboost_val_pred = catboost_model.predict_proba(X_val)[:, 1]
catboost_auc = roc_auc_score(y_val, catboost_val_pred)
print(f"Final CatBoost validation AUC: {catboost_auc:.4f}")

[I 2025-11-12 10:07:22,459] A new study created in memory with name: no-name-52ea6ed1-feab-45a5-b98e-c6ddc224efee


Optimizing CatBoost hyperparameters...


Best trial: 0. Best value: 0.908986:   5%|▌         | 1/20 [00:18<05:56, 18.75s/it]

[I 2025-11-12 10:07:41,226] Trial 0 finished with value: 0.9089859855849388 and parameters: {'iterations': 437, 'depth': 10, 'learning_rate': 0.22227824312530747, 'l2_leaf_reg': 6.387926357773329, 'random_strength': 0.24041677639819287, 'bagging_temperature': 0.2403950683025824, 'border_count': 45}. Best is trial 0 with value: 0.9089859855849388.


Best trial: 1. Best value: 0.910597:  10%|█         | 2/20 [00:40<06:06, 20.36s/it]

...

[I 2025-11-12 10:11:15,119] Trial 17 finished with value: 0.918470487077188 and parameters: {'iterations': 694, 'depth': 4, 'learning_rate': 0.18725103746238383, 'l2_leaf_reg': 1.6241128644754506, 'random_strength': 0.6009535996389941, 'bagging_temperature': 0.9886943731160998, 'border_count': 125}. Best is trial 4 with value: 0.9214267505100445.


Best trial: 4. Best value: 0.921427:  95%|█████████▌| 19/20 [04:06<00:14, 14.76s/it]

[I 2025-11-12 10:11:29,023] Trial 18 finished with value: 0.9204667375722642 and parameters: {'iterations': 536, 'depth': 5, 'learning_rate': 0.1423294288603213, 'l2_leaf_reg': 2.9136732663727702, 'random_strength': 0.7608796812995494, 'bagging_temperature': 0.4498922753077476, 'border_count': 232}. Best is trial 4 with value: 0.9214267505100445.


Best trial: 4. Best value: 0.921427: 100%|██████████| 20/20 [04:17<00:00, 12.85s/it]



[I 2025-11-12 10:11:39,533] Trial 19 finished with value: 0.9178878623699516 and parameters: {'iterations': 348, 'depth': 7, 'learning_rate': 0.06511842795285187, 'l2_leaf_reg': 8.663656475710512, 'random_strength': 0.25901840658144304, 'bagging_temperature': 0.9061604145189799, 'border_count': 197}. Best is trial 4 with value: 0.9214267505100445.
Best CatBoost AUC: 0.9214
Best CatBoost params: {'iterations': 633, 'depth': 4, 'learning_rate': 0.1861880070514171, 'l2_leaf_reg': 2.5347171131856236, 'random_strength': 0.15854643368675156, 'bagging_temperature': 0.953996983528, 'border_count': 248}


# TabNet (Attention-Based Deep Learning)

In [ ]:
# Train TabNet Model with Attention Mechanism

def train_tabnet_model():
    """Train TabNet - Attention-based neural network for tabular data"""
    print("🎯 Training TabNet with attention mechanism...")
    
    # Prepare data (TabNet needs numpy arrays)
    X_train_np = X_train.values
    X_val_np = X_val.values
    X_test_np = X_test.values
    y_train_np = y_train.values
    y_val_np = y_val.values
    
    # TabNet configuration
    tabnet_params = {
        "n_d": 64,  # Width of decision prediction layer
        "n_a": 64,  # Width of attention embedding
        "n_steps": 5,  # Number of sequential attention steps
        "gamma": 1.5,  # Coefficient for feature reusage
        "n_independent": 2,  # Independent GLU layers
        "n_shared": 2,  # Shared GLU layers
        "momentum": 0.3,  # Batch norm momentum
        "mask_type": "entmax",  # Attention mask type
        "lambda_sparse": 1e-4,  # Sparsity loss coefficient
        "seed": 42,
        "verbose": 0
    }
    
    # Initialize and train
    tabnet_model = TabNetClassifier(**tabnet_params)
    tabnet_model.fit(
        X_train_np, y_train_np,
        eval_set=[(X_val_np, y_val_np)],
        eval_metric=['auc'],
        max_epochs=100,
        patience=20,
        batch_size=1024,
        virtual_batch_size=128,
        num_workers=0,
        drop_last=False
    )
    
    # Evaluate
    val_pred_tabnet = tabnet_model.predict_proba(X_val_np)[:, 1]
    test_pred_tabnet = tabnet_model.predict_proba(X_test_np)[:, 1]
    tabnet_auc = roc_auc_score(y_val_np, val_pred_tabnet)
    
    print(f"✅ TabNet AUC: {tabnet_auc:.4f}")
    
    # Feature importance from attention
    feature_importances = tabnet_model.feature_importances_
    feature_imp_df = pd.DataFrame({
        'feature': X.columns,
        'importance': feature_importances
    }).sort_values('importance', ascending=False)
    
    print("\nTop 10 Features by TabNet Attention:")
    print(feature_imp_df.head(10).to_string(index=False))
    
    return tabnet_model, test_pred_tabnet, tabnet_auc, val_pred_tabnet

# Train TabNet
tabnet_model, tabnet_predictions, tabnet_auc, tabnet_val_pred = train_tabnet_model()

print("\n📊 Model Comparison:")
print(f"CatBoost:  {catboost_auc:.4f} AUC")
print(f"TabNet:    {tabnet_auc:.4f} AUC")

🎯 TRAINING TABNET - Attention-Based Deep Learning for Tabular Data
🎯 Training TabNet Model...
TabNet uses sequential attention mechanism to select important features
Training TabNet with attention mechanism...
Stop training because you reached max_epochs = 100 with best_epoch = 99 and best_val_0_auc = 0.913
✅ TabNet AUC: 0.9130

Top 10 Features by TabNet Attention:
                feature  importance
      sqrt_credit_score    0.411894
   debt_to_income_ratio    0.359061
      employment_status    0.183223
credit_score_normalized    0.031117
          annual_income    0.011878
            loan_amount    0.002801
       income_stability    0.000027
                 gender    0.000000
         marital_status    0.000000
        education_level    0.000000

📊 PERFORMANCE COMPARISON:
Original CatBoost:  0.9214 AUC
TabNet Model:       0.9130 AUC
📉 TabNet is 8.4 points behind CatBoost
CatBoost remains the best model


In [ ]:
# Ensemble: Combine CatBoost and TabNet

print("🎯 Creating Hybrid Ensemble: CatBoost + TabNet")
print("=" * 60)

# Find optimal ensemble weights
best_ensemble_auc = 0
best_weight = 0

for catboost_weight in np.arange(0.5, 1.0, 0.05):
    tabnet_weight = 1 - catboost_weight
    ensemble_val = catboost_weight * catboost_val_pred + tabnet_weight * tabnet_val_pred
    ensemble_auc = roc_auc_score(y_val, ensemble_val)
    
    if ensemble_auc > best_ensemble_auc:
        best_ensemble_auc = ensemble_auc
        best_weight = catboost_weight

print(f"Best Ensemble: {best_weight:.0%} CatBoost + {1-best_weight:.0%} TabNet")
print(f"Ensemble AUC: {best_ensemble_auc:.4f}")

# Create ensemble predictions
catboost_test_pred = catboost_model.predict_proba(X_test)[:, 1]
hybrid_predictions = best_weight * catboost_test_pred + (1-best_weight) * tabnet_predictions

print("\n📊 Final Model Rankings:")
print(f"1. CatBoost (Optimized):  {catboost_auc:.4f} AUC")
print(f"2. Hybrid Ensemble:       {best_ensemble_auc:.4f} AUC")
print(f"3. TabNet:                {tabnet_auc:.4f} AUC")

🎯 Creating Hybrid Ensemble: CatBoost + TabNet
Testing different ensemble weights...

Best Ensemble Configuration:
CatBoost weight: 0.95
TabNet weight: 0.05
Ensemble AUC: 0.9213

✅ Final hybrid submission created!
File: submission_final_hybrid.csv

📊 FINAL MODEL COMPARISON:
1. Original CatBoost:    0.9214 AUC
2. TabNet:               0.9130 AUC
3. Hybrid Ensemble:      0.9213 AUC

✅ CatBoost remains the champion at 0.9214 AUC


# Generate Predictions & Create Submission

In [ ]:
# Create final predictions with post-processing

def apply_post_processing(predictions, target_mean):
    """Apply calibration to match training distribution"""
    processed = np.clip(predictions, 0.002, 0.998)
    current_mean = processed.mean()
    adjustment = target_mean / current_mean
    processed = processed * adjustment
    processed = np.clip(processed, 0.001, 0.999)
    return processed

# Get training target mean
train_target_mean = y.mean()

# Process predictions
hybrid_processed = apply_post_processing(hybrid_predictions, train_target_mean)
catboost_test_pred = catboost_model.predict_proba(X_test)[:, 1]
catboost_processed = apply_post_processing(catboost_test_pred, train_target_mean)

# Create submissions
submission_hybrid = pd.DataFrame({
    'id': test_processed['id'],
    'loan_paid_back': hybrid_processed
})

submission_catboost = pd.DataFrame({
    'id': test_processed['id'],
    'loan_paid_back': catboost_processed
})

# Save files
submission_hybrid.to_csv('submission_hybrid.csv', index=False)
submission_catboost.to_csv('submission_catboost.csv', index=False)

print("✅ Submission files created!")
print("\n📁 Files:")
print(f"  • submission_catboost.csv (CatBoost only - {catboost_auc:.4f} AUC)")
print("  • submission_hybrid.csv (CatBoost + TabNet ensemble)")

print("\n📊 Prediction Statistics:")
print(f"Training mean: {train_target_mean:.4f}")
print(f"CatBoost mean: {catboost_processed.mean():.4f}")
print(f"Hybrid mean:   {hybrid_processed.mean():.4f}")

print("\n🎯 Recommended: submission_catboost.csv")
print(f"   (Best validation AUC: {catboost_auc:.4f})")

# Results Summary

## Final Model Performance

| Model | Validation AUC |
|-------|---------------|
| **CatBoost (Optimized)** | **0.9214** |
| Hybrid Ensemble | 0.9213 |
| TabNet | 0.9130 |

## Top 5 Features (CatBoost)
1. Employment Status (54.9%)
2. Debt-to-Income Ratio (14.1%)
3. Credit-Employment Combo (6.1%)
4. Sqrt Credit Score (6.0%)
5. Risk Score (5.8%)

## Submission Files
- `submission_catboost.csv` - **Recommended** (0.9214 AUC)
- `submission_hybrid.csv` - Ensemble approach

See README.md for full methodology and insights.